In [ ]:
# Utilização da técnica SMOTE para oversampling das imagens

In [1]:
import numpy as np
import pandas as pd
import cv2
import random
import os
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

In [174]:
path1 = os.getcwd()+'//..//features//'
path2 = os.getcwd()+'//..//features2//'

In [256]:
data1 = pd.read_csv(path1+'all_features.csv', sep=',', index_col=0)
data2 = pd.read_csv(path2+'all_features.csv', sep=',', index_col=0)

sst1 = pd.read_csv(path1+'/../reference_values/primeiros_dados_palmer_sst.csv', sep=',', index_col=0)
firmeza1 = pd.read_csv(path1+'/../reference_values/primeiros_dados_palmer_firmeza.csv', sep=',', index_col=0)
time1 = pd.read_csv(path1+'/../reference_values/primeiros_dados_palmer_time.csv', sep=',', index_col=0)
sst2 = pd.read_csv(path2+'/../reference_values/ultimos_dados_palmer_sst.csv', sep=',', index_col=0)
firmeza2 = pd.read_csv(path2+'/../reference_values/ultimos_dados_palmer_firmeza.csv', sep=',', index_col=0)
time2 = pd.read_csv(path2+'/../reference_values/ultimos_dados_palmer_time.csv', sep=',', index_col=0)

In [176]:
to_append = data1.iloc[600:]
sst_to_append = sst1.iloc[600:]
firmeza_to_append = firmeza1.iloc[600:]
time_to_append = time1.iloc[600:]

In [129]:
s1 = set(data1.columns.values)
s2 = set(data2.columns.values)

In [ ]:
s1.difference(s2)

In [177]:
new_data = data2.append(to_append)
new_sst = sst2.append(sst_to_append)
new_firmeza = firmeza2.append(firmeza_to_append)
new_time = time2.append(time_to_append)

In [178]:
new_data['sst'] = new_sst
new_data['firmeza'] = new_firmeza
new_data['time'] = new_time

In [179]:
k = 5
N = 200

In [180]:
nums = np.arange(1651,1651+int(new_data.shape[0]*(N/100)))
indexes = [str(num)+'_synthetic' for num in nums]

In [195]:
df = smote(new_data, k, N, indexes, 1651, ['time', 'sst', 'firmeza'])

[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 160 166 162]
[175 167 1

In [194]:
def smote(data, k, N, indexes, i, targets):
    N = (int)(N/100)
    n_samples = data.shape[0]
    features = data.drop(columns=targets)
    synthetic = pd.DataFrame(index=indexes,columns=data.columns.values)
    
    neigh = NearestNeighbors(n_neighbors=k+1)
    neigh.fit(features)
    
#     def treat_targets(neighbors, sample, targets):
#         sm = 0
#         calc_targ = []
        
#         for target in targets:
#             sm = 0
#             for neigh in range(neighbors):
#                 sm = sm + (sample[target]-features.iloc[neigh][target])
    
    def normalize_features():
        return features.apply(lambda x: x/np.max(abs(x)))
    
#     def post_normalize_features():
    def get_neighbors(sample):
        return neigh.kneighbors([sample], return_distance=False)[0][1:]
        
    def populate(norm, neighbors, sample, i):
        for __ in range(N):
            rnd = random.randint(0,k-1)
            
            diff = features.iloc[neighbors[rnd]] - features.iloc[sample]
            gap = random.uniform(0,1)
            feature_row = features.iloc[sample] + diff*gap
            
            synthetic_target = data.iloc[sample][targets]
            synthetic_sample = pd.Series(data=feature_row, index=data.columns.values, name=str(i)+'_synthetic')
#             print(synthetic_sample)
#             print(feature_row)
            
            for target in targets:
                synthetic_sample[target] = data.iloc[sample][target]
            
            synthetic.loc[str(i)+'_synthetic'] = synthetic_sample
            
            i=i+1
        return i
        
    norm = normalize_features()
    
    for sample in range(n_samples):
        neighbors = get_neighbors(norm.iloc[sample])
        print(neighbors)
        i = populate(norm, neighbors, sample, i)
    
    return synthetic

In [197]:
df.to_csv('synthetic_data.csv', sep=',')

In [257]:
data1['sst'] = sst1
data1['firmeza'] = firmeza1
data1['time'] = time1
data2['sst'] = sst2
data2['firmeza'] = firmeza2
data2['time'] = time2

In [254]:
all_data = pd.concat([data1, data2, df], axis=1)

In [211]:
600+240+480

1320